In [1]:
import numpy as np
from numba import njit, int64, float64, int32
from scipy import sparse
import timeit

In [2]:
@njit((float64[:], int64[:], int64[:], int64), fastmath=True)
def create_sparse_matrix(data, rows, cols, nrows):
    nnz = len(data)

    indptr = np.zeros(nrows, dtype=int32)
    indices = np.zeros(nnz, dtype=int32)
    data_csr = np.zeros(nnz, dtype=float64)

    for i in range(nnz):
        indptr[rows[i] + 1] += 1
        indices[i] = cols[i]
        data_csr[i] = data[i]

    indptr = np.cumsum(indptr)
    return data_csr, indptr, indices


def format_sparse_matrix(data, rows, cols, nrows, ncols):
    sparse_matrix = create_sparse_matrix(data, rows, cols, nrows)
    dense_matrix = np.zeros((nrows, ncols))

    for i in range(len(data)):
        row = rows[i]
        col = cols[i]
        value = data[i]
        dense_matrix[row, col] = value

    return dense_matrix

data = np.array([0.1, 0.2, 0.3])
rows = np.array([0, 1, 2])
cols = np.array([2, 1, 0])
nrows = 3
ncols = 3

dense_matrix = format_sparse_matrix(data, rows, cols, nrows, ncols)
print(dense_matrix)

sparse_matrix = create_sparse_matrix(data, rows, cols, nrows)
print(sparse_matrix)
sparse_matrix_scipy = sparse.coo_matrix((data, (rows, cols)), shape=(nrows, ncols))
print(sparse_matrix_scipy)

[[0.  0.  0.1]
 [0.  0.2 0. ]
 [0.3 0.  0. ]]
(array([0.1, 0.2, 0.3]), array([0, 1, 2]), array([2, 1, 0], dtype=int32))
  (0, 2)	0.1
  (1, 1)	0.2
  (2, 0)	0.3


In [3]:
# Benchmark test
#def benchmark():
num_iterations = 100000
print(f"Number of iterations: {num_iterations}")

# Data, rows, cols, and shape for sparse matrix construction
data = np.random.rand(1000)
rows = np.random.randint(0, 100, 1000)
cols = np.random.randint(0, 100, 1000)
shape = (1000, 1000)

# Benchmark construction using scipy.sparse.coo_matrix
def benchmark_scipy():
    for _ in range(num_iterations):
        sparse_matrix = sparse.coo_matrix((data, (rows, cols)), shape=shape)

# Benchmark construction using NumPy and Numba
def benchmark_numpy_numba():
    for _ in range(num_iterations):
        sparse_matrix = create_sparse_matrix(data, rows, cols, shape[0])

# Run the benchmarks
time_scipy = timeit.timeit(benchmark_scipy, number=1)
time_numpy_numba = timeit.timeit(benchmark_numpy_numba, number=1)

# Print the results
print("Benchmark Results:")
print("scipy.sparse.coo_matrix:", time_scipy)
print("NumPy and Numba:", time_numpy_numba)
# Calculate the speedup
speedup = time_scipy / time_numpy_numba
print(f"Speedup: {speedup:.2f}x")

Number of iterations: 100000
Benchmark Results:
scipy.sparse.coo_matrix: 8.888024165999923
NumPy and Numba: 0.6421734119999201
Speedup: 13.84x
